In [25]:
# EfficientNet-B5 문서 이미지 분류 모델
# Train 데이터: 1570장 (깨끗한 이미지)
# Test 데이터: 3140장 (회전, 플립, 노이즈가 심한 이미지)
# 클래스: 17개 (불균형 데이터)

import os
import random
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import wandb
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [62]:
# ================================
# 14. 메인 실행
# ================================
if __name__ == '__main__':
    print('Starting training with K-Fold Cross Validation...')
    train_with_kfold()
    
    print('\nStarting test prediction with TTA...')
    predict_with_tta()
    
    if config.use_wandb:
        wandb.finish()
    
    print('Done!')

Starting training with K-Fold Cross Validation...

Fold 1/5


model.safetensors:   0%|          | 0.00/173M [00:00<?, ?B/s]

Validating: 100%|██████████| 20/20 [00:04<00:00,  4.74it/s]


Epoch 1/50
LR: 0.000200
Train Loss: 1.3964, Train Acc: 57.96%
Valid Loss: 0.5635, Valid Acc: 82.80%
✓ Best model saved! (Val Acc: 82.80%)


Validating: 100%|██████████| 20/20 [00:04<00:00,  4.73it/s]


...

Epoch 50/50
LR: 0.000002
Train Loss: 0.0040, Train Acc: 99.92%
Valid Loss: 0.1994, Valid Acc: 93.31%

Fold 5 Best Validation Accuracy: 95.22%

Cross-Validation Results:
Fold 1: 95.86%
Fold 2: 95.86%
Fold 3: 95.54%
Fold 4: 92.68%
Fold 5: 95.22%
Mean CV Accuracy: 95.03% ± 1.20%

Starting test prediction with TTA...


Predicting with TTA: 100%|██████████| 3140/3140 [34:39<00:00,  1.51it/s]

Submission file saved!


cv_mean_accuracy,▁
cv_std_accuracy,▁
epoch,▃▃▄▄▅▇▇▇▁▁▆▇▇▇██▁▂▂▂▅▅▅▅▆▇█▁▄▄▆▆▇▂▂▄▅▅▇█
fold_1/learning_rate,▂▄▅▇███████▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁
fold_1/train_acc,▁▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇█▇████████████████████
fold_1/train_loss,█▅▄▄▄▃▂▃▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
fold_1/val_acc,▁▃▄▂▂▅▅▇▅▂▇▅▆▆▇▇▇█▅▇█▇▇▇▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇
fold_1/val_loss,▇▅▃█▅▃▁▃▆▂▃▁▂▁▁▂▁▃▁▁▁▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
fold_2/learning_rate,▂▄▅▇██████▇▇▇▇▇▆▆▆▆▆▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁
fold_2/train_acc,▁▄▅▅▅▆▇▇▇▇▇▇▇▇▇▇▇▇▇█████████████████████
+18,...


Done!
